### word2vec 的实现

In [1]:
import collections
import math
import random
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data

#### 1.1 读取数据

In [3]:
# 确保数据集存在
assert "ptb.train.txt" in os.listdir("../data/ptb")

PTB_DATA_PATH = "../data/ptb/"

with open(PTB_DATA_PATH + "ptb.train.txt", 'r') as f:
    # 读取所有行
    lines = f.readlines()
    datasets = [line.split() for line in lines]

print("句子总行数 : %d" % (len(datasets)))

句子总行数 : 42068


#### 1.2 打印数据集

对于数据集的前3个句子，打印每个句子的词数和前5个词。这个数据集中句尾符为\<eos>，生僻词全用\<unk>表示，数字则被替换成了"N"。

In [4]:
for line in datasets[:3]:
    print("# token : ", len(line), line[:5])

# token :  24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
# token :  15 ['pierre', '<unk>', 'N', 'years', 'old']
# token :  11 ['mr.', '<unk>', 'is', 'chairman', 'of']


#### 1.3 建立词语索引

- 为了计算简单，我们只保留在数据集中至少出现5次的词。

In [13]:
# [word for line in datasets for word in line] 写法等价于下面的方法
# for line in datasets:
#     for word in line:
#         print(word)

words = [word for line in datasets for word in line]
# {'the': 50770} => {单词:词频}
counter = collections.Counter(words)
print(len(words))
print(len(counter))

# 过滤掉词频小于5的单词
counter = dict(filter(lambda x: x[1] > 5, counter.items()))
print(len(counter))

887521
9999
9582


#### 1.4 将数据集转换为索引

- 上一步建立词语索引后, 我们使用词语索引将数据集中的句子中的单词转换为索引
- 总结 :
    1. 使用 collections.Counter 统计词频, 传入单词表, 生成  {'the': 50770} => {单词:词频} 这样的字典
    2. 过滤掉词频过小的单词
    3. 对单词建立索引, 使用 enumerate(word_all)
    4. 遍历数据集, 得到每一个句子, 然后再遍历句子得到单词, 使用单词索引表将单词转换为索引, 然后存储到句子索引数组中
    5. 然后将句子索引数组放到新数据集中, 例子如下 : 可以看到数组中有三个索引数组, 每一个数组都代表一个句子
    6. [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  [14, 1, 15, 16, 19, 20, 21],  [22, 1, 2, 3,10, 11, 12, 17, 31, 32, 33, 34]]

In [29]:
# 获取所有的单词
word_all = [word for word, count in counter.items()]
print(word_all[:2])

# 对单词建立索引
# {'pierre': 0, '<unk>': 1, 'N': 2, 'years': 3}
word_to_idx = {word: index for index, word in enumerate(word_all)}

# 将数据集转换为词索引
dataset_idx = []
# 将每一个句子中所有的单词转换为索引, 然后放到一个数组里, 然后再把这个数组放到整个数据集, 数据集以数组为单位
for line in datasets:
    line_idx = []
    for word in line:
        # 如果单词在索引表中
        if word in word_to_idx:
            line_idx.append(word_to_idx[word])
    if len(line_idx) > 0:
        dataset_idx.append(line_idx)
print(dataset_idx[:3])
num_words = sum([len(line) for line in dataset_idx])
print(num_words)

['pierre', '<unk>']
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 2], [14, 1, 15, 16, 17, 1, 18, 7, 19, 20, 21], [22, 1, 2, 3, 4, 23, 24, 16, 17, 25, 26, 27, 28, 29, 30, 10, 11, 12, 17, 31, 32, 33, 34]]
885720


#### 1.5 二次采样

- [二次采样公式](https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter10_natural-language-processing/10.3_word2vec-pytorch?id=_10312-%e4%ba%8c%e6%ac%a1%e9%87%87%e6%a0%b7)
1. 文本数据中一般会出现一些高频词，如英文中的"the""a"和"in"。通常来说，在一个背景窗口中，一个词（如"chip"）和较低频词（如"microprocessor"）
2. 同时出现比和较高频词（如“the”）同时出现对训练词嵌入模型更有益。
3. 因此，训练词嵌入模型时可以对词进行二次采样 [2]。 具体来说，数据集中每个被索引词将有一定概率被丢弃

In [34]:
# 定义二次采样函数
def discard(idx):
    # f(w) > t , t = 1e-4
    return (1 - math.sqrt(1e-4 / (counter[word_all[idx]] / num_words))) > random.uniform(0, 1)


# 进行二次采样, 转换成索引的数据集进行遍历, 然后根据词频决定去除哪些词
subsampled_dataset = []
for line in dataset_idx:
    word_idxs = []
    for word_idx in line:
        if not discard(word_idx):
            word_idxs.append(word_idx)
    subsampled_dataset.append(word_idxs)

print("word => %d" % sum([len(line) for line in subsampled_dataset]))

word => 373961


- 统计某一个单词的采样率

In [39]:
def compare_counts(word):
    before_count = 0
    for line in dataset_idx:
        before_count += line.count(word_to_idx[word])

    after_count = 0
    for line in subsampled_dataset:
        after_count += line.count(word_to_idx[word])

    print("before %d " % before_count)
    print("after %d " % after_count)


compare_counts('the')

before 50770 
after 2151 


In [40]:
compare_counts('join')

before 45 
after 45 


#### 1.6 提取中心词和背景词

我们将与中心词距离不超过背景窗口大小的词作为它的背景词。下面定义函数提取出所有中心词和它们的背景词。它每次在整数1和max_window_size（最大背景窗口）之间随机均匀采样一个整数作为背景窗口大小。

In [ ]:
# 定义提取背景词和中心词的代码

def get_center_context(dataset, max_windows_size):
    center, context = [], []
    # 单个句子中的词汇必须大于2才能组成 中心词+背景词
    for line in dataset:
        if len(line) < 2:
            continue